In [7]:
import fastmap
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.svm
import pandas as pd

In [16]:
nc_all=pd.read_csv('/Users/alexdseo/Documents/Research Intern/RACARA/GLDS-120_rna_seq_Normalized_Counts.csv')

In [17]:
nc_all.rename( columns={'Unnamed: 0' :'RNA_seq'}, inplace=True )

In [18]:
# Reorder columns
nc_all2=nc_all[['RNA_seq', 'Atha_Col-0_root_FLT_Alight_Rep1_GSM2493777_Day13',
                'Atha_Col-0_root_FLT_Alight_Rep2_GSM2493778_Day13',
                'Atha_Col-0_root_FLT_Alight_Rep3_GSM2493779_Day13',
                'Atha_Col-0-PhyD_root_FLT_Alight_Rep1_GSM2493783_Day13',
                'Atha_Col-0-PhyD_root_FLT_Alight_Rep2_GSM2493784_Day13',
                'Atha_Col-0-PhyD_root_FLT_Alight_Rep3_GSM2493785_Day13',
                'Atha_Ws_root_FLT_Alight_Rep1_GSM2493780_Day13',
                'Atha_Ws_root_FLT_Alight_Rep2_GSM2493781_Day13',
                'Atha_Ws_root_FLT_Alight_Rep3_GSM2493782_Day13',
                'Atha_Col-0_root_FLT_dark_Rep1_GSM2493786_Day13',
                'Atha_Col-0_root_FLT_dark_Rep2_GSM2493787_Day13',
                'Atha_Col-0_root_FLT_dark_Rep3_GSM2493788_Day13',
                'Atha_Col-0-PhyD_root_FLT_dark_Rep1_GSM2493792_Day13',
                'Atha_Col-0-PhyD_root_FLT_dark_Rep2_GSM2493793_Day13',
                'Atha_Col-0-PhyD_root_FLT_dark_Rep3_GSM2493794_Day13',
                'Atha_Ws_root_FLT_dark_Rep1_GSM2493789_Day13',
                   'Atha_Ws_root_FLT_dark_Rep2_GSM2493790_Day13',
                   'Atha_Ws_root_FLT_dark_Rep3_GSM2493791_Day13',
                'Atha_Col-0_root_GC_Alight_Rep1_GSM2493759_Day13',
                   'Atha_Col-0_root_GC_Alight_Rep2_GSM2493760_Day13',
                   'Atha_Col-0_root_GC_Alight_Rep3_GSM2493761_Day13',
                'Atha_Col-0-PhyD_root_GC_Alight_Rep1_GSM2493765_Day13',
                   'Atha_Col-0-PhyD_root_GC_Alight_Rep2_GSM2493766_Day13',
                   'Atha_Col-0-PhyD_root_GC_Alight_Rep3_GSM2493767_Day13',
                                   'Atha_Ws_root_GC_Alight_Rep1_GSM2493762_Day13',
                   'Atha_Ws_root_GC_Alight_Rep2_GSM2493763_Day13',
                   'Atha_Ws_root_GC_Alight_Rep3_GSM2493764_Day13',
                   'Atha_Col-0_root_GC_dark_Rep1_GSM2493768_Day13',
                   'Atha_Col-0_root_GC_dark_Rep2_GSM2493769_Day13',
                   'Atha_Col-0_root_GC_dark_Rep3_GSM2493770_Day13',
                   'Atha_Col-0-PhyD_root_GC_dark_Rep1_GSM2493774_Day13',
                   'Atha_Col-0-PhyD_root_GC_dark_Rep2_GSM2493775_Day13',
                   'Atha_Col-0-PhyD_root_GC_dark_Rep3_GSM2493776_Day13',
                'Atha_Ws_root_GC_dark_Rep1_GSM2493771_Day13',
                   'Atha_Ws_root_GC_dark_Rep2_GSM2493772_Day13',
                   'Atha_Ws_root_GC_dark_Rep3_GSM2493773_Day13']]

In [19]:
###Data reconstruction, Make more variables and set the output(Y) variable
###You can not use melt() from pandas since there are 3 rep for each segment

###replicate each row 12 times for same RNA_seq
newnc = pd.DataFrame(np.repeat(nc_all2.values,2,axis=0))
#newnc.colums = nc_all.columns

###Create Light setting variable

###create Location variable
LC = pd.DataFrame(
{"Location" : ['FLT', 'GC']})
LC = pd.concat([LC]*len(nc_all2)).reset_index().drop(columns=['index'])

###Transpose fpkm values (3 values)
ncworna = nc_all2.drop(columns = ['RNA_seq'])
D = pd.DataFrame(columns=['Rep1_c_l', 'Rep2_c_l', 'Rep3_c_l',
                         'Rep1_p_l', 'Rep2_p_l', 'Rep3_p_l',
                         'Rep1_w_l', 'Rep2_w_l', 'Rep3_w_l',
                         'Rep1_c_d', 'Rep2_c_d', 'Rep3_c_d',
                         'Rep1_p_d', 'Rep2_p_d', 'Rep3_p_d',
                         'Rep1_w_d', 'Rep2_w_d', 'Rep3_w_d'])

for i in range(len(ncworna)):
    s = pd.Series(ncworna.iloc[i]) #Make each row a pandas series
    a = np.reshape(s.to_numpy(),(2,18)) #Change Series to numpy and reshape it
    #Make each reshaped arrays into dataframe, this one dataframe represent one RNA sequence
    d = pd.DataFrame(a, columns=['Rep1_c_l', 'Rep2_c_l', 'Rep3_c_l',
                         'Rep1_p_l', 'Rep2_p_l', 'Rep3_p_l',
                         'Rep1_w_l', 'Rep2_w_l', 'Rep3_w_l',
                         'Rep1_c_d', 'Rep2_c_d', 'Rep3_c_d',
                         'Rep1_p_d', 'Rep2_p_d', 'Rep3_p_d',
                         'Rep1_w_d', 'Rep2_w_d', 'Rep3_w_d'])
    D = pd.concat([D,d]) #concat each dataframe

D = D.reset_index().drop(columns=['index'])#Reset index
###Add and drop variables
newnc = newnc.drop(columns=range(1,37))

newnc["Location"] = LC

newnc = pd.concat([newnc, D], axis = 1)


#Rename column
newnc = newnc.rename(columns = {0 : "RNA_seq"})
#Overview of newly constructed dataset
newnc.head()

,RNA_seq,Location,Rep1_c_l,Rep2_c_l,Rep3_c_l,Rep1_p_l,Rep2_p_l,Rep3_p_l,Rep1_w_l,Rep2_w_l,Rep3_w_l,Rep1_c_d,Rep2_c_d,Rep3_c_d,Rep1_p_d,Rep2_p_d,Rep3_p_d,Rep1_w_d,Rep2_w_d,Rep3_w_d
0,AT1G01010,FLT,247.594834,221.299634,328.875256,253.884065,275.393343,486.247724,204.291408,183.830544,194.349657,424.846381,147.993850,277.181771,398.108571,104.632768,85.671819,115.873687,309.807212,320.965086
1,AT1G01010,GC,114.903911,138.527045,105.631417,242.522087,412.252180,210.848379,120.029155,264.939077,191.422436,144.663018,201.741865,285.221585,355.864675,190.024910,319.742539,66.843506,155.468283,198.730734
2,AT1G01020,FLT,582.003386,525.522545,557.716573,662.721977,620.405261,619.800417,460.643087,727.144235,531.944938,463.444481,713.777447,961.748406,581.378782,749.909700,688.502365,496.329434,569.702452,454.006600
3,AT1G01020,GC,673.520214,621.787935,559.369964,748.231809,537.976709,739.321906,645.973410,439.206255,522.399132,656.624218,627.928204,633.934741,666.197531,630.215736,621.323888,488.282906,517.722067,516.581267
4,AT1G01030,FLT,64.785419,54.875704,110.764910,42.589529,58.928829,85.041329,74.235930,153.874754,83.046646,49.956995,80.949832,162.368184,97.235756,182.950314,249.211228,43.760519,105.956723,136.694433


In [20]:
#Applying models using newly constructed labeled dataset

#Label Encoding - One hot encoding on categorical predictors for modeling purpose
#ncwd3 = pd.get_dummies(newnc, columns = ["Genotype"],drop_first =True)
#ncwd3 = pd.get_dummies(ncwd, columns = ['Light_Setting'])
ncwd4 = newnc
clean_target = {"Location": {"FLT": 1, "GC": 0}}
ncwd4.replace(clean_target, inplace = True)

#ncwd.head()

In [21]:
ncwd4= ncwd4[['RNA_seq','Rep1_c_l', 'Rep2_c_l', 'Rep3_c_l', 'Rep1_p_l', 'Rep2_p_l', 'Rep3_p_l', 'Rep1_w_l', 'Rep2_w_l', 'Rep3_w_l', 
              'Rep1_c_d', 'Rep2_c_d', 'Rep3_c_d', 'Rep1_p_d', 'Rep2_p_d', 'Rep3_p_d', 'Rep1_w_d', 'Rep2_w_d', 'Rep3_w_d','Location']]

In [114]:
X_train = ncwd4.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]].values
y_train = ncwd4['Location'].values

In [115]:
X_train=np.reshape(X_train,(len(X_train),2,9))

In [116]:
def correlation_distance(a, b, axis=-1):
        '''
        Compute the pair-wise correlation distance matrix.
        '''
        xcorr = correlate(a, b, axis=axis)
        xcorr = np.abs(xcorr)
        xcorr = np.nanmean(xcorr, axis=-2)
        xcorr = np.max(xcorr, axis=-1)
        xcorr = np.clip(xcorr, 0, 1)
        
        return 1 - xcorr
    

def correlate(a, b, axis=-1):
    A = np.fft.rfft(a, axis=axis)
    B = np.fft.rfft(b, axis=axis)
    X = A * np.conj(B)
    x = np.fft.irfft(X)
    x = np.fft.fftshift(x, axes=axis)
    norm = np.sqrt(
        a.shape[-1] * np.var(a, axis=axis)
        *
        b.shape[-1] * np.var(b, axis=axis)
    )
    norm = norm[..., np.newaxis]

    return np.nan_to_num(x / norm, neginf=0, posinf=0)

In [117]:
def generic_distance(a, b, axis=-1):
    # Build the output array with broadcasting rules applied.
    shape = np.broadcast_shapes(a.shape, b.shape)
    axis = axis if axis > -1 else len(shape) + axis
    shape = shape[:axis] + shape[axis+1:]
    output = np.empty(shape)
    n_dim = output.ndim

    # Loop over elements and compute distances serially.
    for ijk in np.ndindex(*output.shape):
        ijk_a = tuple([ijk[i] if a.shape[i] != 1 else 0 for i in range(len(ijk))])
        ijk_b = tuple([ijk[i] if b.shape[i] != 1 else 0 for i in range(len(ijk))])
        output[ijk] = dist(a[ijk_a], b[ijk_b])
        
    return output
        
def dist(a, b, axis=-1):
    '''
    Return the distance between objects a and b.
    '''
    return np.linalg.norm(a - b, axis=axis)

In [118]:
class FastMap(fastmap.FastMapABC):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._distance_func = correlation_distance

In [119]:
n_dim = 2 # The number of dimensions for the Euclidean embedding.
fastmapsvm = sklearn.pipeline.Pipeline([
    ('fastmap', FastMap(n_dim)),
    ('scaler', sklearn.preprocessing.StandardScaler()),
    ('svc', sklearn.svm.SVC())
])

In [75]:
# with h5py.File('data/ridgecrest.hdf5', mode='r') as in_file:
#     X_train = in_file['/X/train'][:]
#     y_train = in_file['/y/train'][:]

#     X_test = in_file['/X/test'][:]
#     y_test = in_file['/y/test'][:]

In [120]:
X_train.shape

(49480, 2, 9)

In [121]:
y_train.shape

(49480,)

In [129]:
fastmapsvm.fit(X_train, y_train)
fastmapsvm.score(X_train, y_train)

/var/folders/6j/lkz2v7js2vq494yrtl79cs0h0000gn/T/ipykernel_71082/1723740042.py:27: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(x / norm, neginf=0, posinf=0)
/var/folders/6j/lkz2v7js2vq494yrtl79cs0h0000gn/T/ipykernel_71082/1723740042.py:27: RuntimeWarning: invalid value encountered in divide
  return np.nan_to_num(x / norm, neginf=0, posinf=0)


0.5067502021018593

In [131]:
# # Compute the embedding of the train data
# W = fastmapsvm[0].transform(X_train)

# plt.close('all')
# fig, ax = plt.subplots()
# for label in range(2):
#     idxs = np.argwhere(y_train == label).flatten()
#     ax.scatter(W[idxs, 0], W[idxs, 1])
# ax.set_aspect(1)
# ax.set_xlabel('$w_0$')
# ax.set_ylabel('$w_1$')